## 2. 웹스크래핑 연습문제


In [ ]:
!pip install requests

In [ ]:
!pip install beautifulsoup4

In [4]:
# reqeusts, bs4 import
import requests
import bs4
# BeautifulSoup 클래스 import
from bs4 import BeautifulSoup

In [5]:
# requests, bs4 버전 확인하기
print(f'requests 버전 = {requests.__version__}')
print(f'bs4 버전 = {bs4.__version__}')

requests 버전 = 2.32.5
bs4 버전 = 4.14.3


### 2-1. Nate 뉴스기사 제목 스크래핑하기 (필수)

In [11]:
import requests
from bs4 import BeautifulSoup

section_dict = {
    '정치': 'n0200',
    '경제': 'n0300',
    '사회': 'n0400',
    '세계': 'n0500',
    'IT/과학': 'n0600',
    '최신': 'n0100',
}

def print_news(section_name):  # 예: print_news('경제')
    mid = section_dict.get(section_name, 'n0100')  # 기본은 최신
    url = f'https://news.nate.com/recent?mid={mid}'
    print(f'{section_name} 뉴스 {url}')

    req_header = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.36'
    }

    return url, req_header


In [12]:
print_news('경제')

경제 뉴스 https://news.nate.com/recent?mid=n0300


('https://news.nate.com/recent?mid=n0300',
 {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.36'})

### 1) Image의 도메인이름이 포함된 url과 src 속성의 img 경로를 합치려면 urljoin 함수를 사용하세요.
    from urllib.parse import urljoin
    


In [13]:
from urllib.parse import urljoin

url = "https://news.nate.com/recent?mid=n0100"
src = "//thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/na/2025/07/23/7408335_high.jpg"

img_url = urljoin(url, src)
print(img_url)


https://thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/na/2025/07/23/7408335_high.jpg


### 2) Image 출력은 Image 클래스와 display 함수를 사용하세요.
    from IPython.display import Image, display


In [14]:
from IPython.display import Image, display

img_url = "https://thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/na/2025/07/23/7408335_high.jpg"
display(Image(url=img_url))


### 3) img 엘리먼트의 존재 여부를 체크하신 후에 src 속성의 이미지를 경로를 추출하기
  => Image 가 없는 뉴스도 있기 때문에 


In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from IPython.display import Image, display

page_url = "https://news.nate.com/recent?mid=n0100"
req_header = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.36"
}

res = requests.get(page_url, headers=req_header, timeout=15)
res.raise_for_status()

soup = BeautifulSoup(res.text, "html.parser")

a_tags = soup.select('a[href*="/view/"]')

seen = set()
count = 0

for a in a_tags:
    href = a.get("href")
    if not href:
        continue

    link = urljoin(page_url, href)
    if link in seen:
        continue

    title = a.get_text(" ", strip=True) or (a.get("title") or "").strip()
    if not title:
        continue

    item = a.find_parent(["li", "article", "div"])  # item이 여기서 정의됨!
    if item is None:
        item = a  # 최악의 경우 a 자체를 item으로 사용

    img_tag = item.select_one("img")
    src = None
    if img_tag is not None:
        src = img_tag.get("src") or img_tag.get("data-src") or img_tag.get("data-original")
        if src:
            src = src.strip()

    print(title)
    print(link)

    if src:
        img_url = urljoin(page_url, src)
        display(Image(url=img_url))
    else:
        print("(이미지 없음)")

    print("-" * 60)

    seen.add(link)
    count += 1
    if count >= 10:
        break


"이름 석자가 죄" 옥주현, 다시 불붙은 '옥장판' 논란…연일 의미심장 메시지 가수 겸 뮤지컬 배우 옥주현이 의미심장한 메시지를 남기며 또 한 번 화제의 중심에 섰다. 옥주현은 9일 자신의 SNS를 통해 뮤지컬 레드북 마지막 공연 소감을 전했다. 그...
https://news.nate.com/view/20260209n39445?mid=n0100


------------------------------------------------------------
'애모가족' 딸도 아동학대 피해자였다…"母, 조카와 비교하며 때려" 고백 (가족지옥) (엑스포츠뉴스 김지영 기자) ‘애모가족’ 어머니가 딸을 끊임없는 비교와 폭행으로 키워왔던 과거가 공개됐다. 9일 방송된 MBC '오은영 리포트-가족 지옥'(이하 '가족 지...
https://news.nate.com/view/20260209n39604?mid=n0100


------------------------------------------------------------
'강성욱 23점' KT, '김효범 감독 지각' 삼성 꺾고 3연패 탈출 [스포츠투데이 이상필 기자] 프로농구 수원 KT가 연장 승부 끝에 서울 삼성을 꺾고 3연패에서 탈출했다. KT는 9일 수원 KT 소닉붐 아레나에서 열린 2025-2026 LG전자 프로...
https://news.nate.com/view/20260209n39603?mid=n0100


------------------------------------------------------------
[부고] 박철우(서울중앙지검장)씨 부친상 [서울=뉴시스] ▲박명록씨 별세, 박철우(서울중앙지검장)씨 부친상 = 9일, 광주광역시 VIP 장례타운 VVIP 301호, 발인 12일 오전 8시 062-521-4444 ▶ 네이버에서 뉴시스 구독하기▶ K-Artprice, 유명 미술작품 가격...
https://news.nate.com/view/20260209n39602?mid=n0100
(이미지 없음)
------------------------------------------------------------
현대캐피탈, KB손해보험 꺾고 1위 탈환 남자 프로배구 현대캐피탈이 KB손해보험을 꺾고 1위에 다시 올랐습니다. 현대캐피탈은 KB손해보험과의 원정 경기에서 37점을 합작한 주포 레오와 허수봉을 앞세워 세트 점수 3대 0으로 완승했습니다. 이로써 현대캐...
https://news.nate.com/view/20260209n39601?mid=n0100
(이미지 없음)
------------------------------------------------------------
[오피셜] 꽈당! 꽈당! 14억 중국 대륙 울었다…'中 슈퍼스타' 구아이링, 2번 넘어지고 女 슬로프스타일 은메달 (엑스포츠뉴스 김현기 기자) "중국과 미국을 넘나든다"는 비판과 논란에 휩싸여 있는 논란의 주인공 구아이링(미국명 에일린 구)이 2026 밀라노·코르티나 동계올림픽 자신...
https://news.nate.com/view/20260209n39567?mid=n0100


------------------------------------------------------------
금산군, 스마트농업 육성지구 명단에 이름 금산군 스마트농업 육성지구 조성사업 조감도. 자료=금산군 제공[금산(충남)=데일리한국 이정석 기자]금산군이 정부 스마트농업 육성지구에 최종 선정되며 전통 농업 중심...
https://news.nate.com/view/20260209n39599?mid=n0100


------------------------------------------------------------
김민석 "능구렁이 같다"는 박충권에 "인신모독, 취소하라" 설전 김민석 국무총리와 국무위원들이 9일 국회 본회의에서 열린 정치ㆍ외교ㆍ통일ㆍ안보 분야 대정부질문에서 의원들 질의에 답하고 있다. 임현동 기자 김민석 국무총리와 탈북...
https://news.nate.com/view/20260209n39584?mid=n0100


------------------------------------------------------------
[동계올림픽] '스노보드 은메달' 김상겸 "끝까지 믿으면 좋은 결과 있을 것" 네 번째 올림픽 만에 메달 획득이번 대회 한국 첫 메달2026 밀라노·코르티나담페초 동계올림픽에서 대한민국 선수단에 첫 메달을 선사한 스노보드 은메달리스트 김상겸(37...
https://news.nate.com/view/20260209n39594?mid=n0100


------------------------------------------------------------
스무 번째 찾아온 전주 '얼굴 없는 천사' 주민센터에 35만원·편지 전달 누적 기부 550만1980원 달해 ‘스무 번째 인사드립니다. 새해 복 많이 받으세요. 늘 감사합니다.’ 익명의 기부자가 남긴 편지와 현금. 전주...
https://news.nate.com/view/20260209n38050?mid=n0100


------------------------------------------------------------


## 2-2. 하나의 네이버 웹툰과 1개의 회차에 대한 Image 다운로드 하기 (필수)


In [21]:
import os
import re
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
from pathlib import Path

REQ_HEADER = {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.36"
}

def _safe_filename(name: str) -> str:
    # 파일/폴더 이름에 위험한 문자 제거
    name = re.sub(r'[\\/:*?"<>|]', "_", name)
    return name.strip()

def _guess_ext_from_url(u: str) -> str:
    path = urlparse(u).path.lower()
    for ext in (".jpg", ".jpeg", ".png", ".webp", ".gif"):
        if path.endswith(ext):
            return ext
    return ".jpg"

def download_one_episode(title, no, url):
    """
    하나의 네이버 웹툰(title) 1개 회차(no)의 이미지들을 다운로드
    - title: 웹툰 제목(폴더명)
    - no: 회차 번호(폴더명)
    - url: 회차 상세 URL
    저장 경로: img/<title>/<no>/
    """
    title = _safe_filename(str(title))
    no = _safe_filename(str(no))

    # 1) 저장 폴더 생성: img/일렉시드/341/
    save_dir = Path("img") / title / no
    save_dir.mkdir(parents=True, exist_ok=True)

    # 2) 회차 페이지 요청
    res = requests.get(url, headers=REQ_HEADER, timeout=20)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, "html.parser")

    # 3) 컷 이미지 찾기 (보통 div.wt_viewer 안에 img들이 있음)
    img_tags = soup.select("div.wt_viewer img")

    # 페이지 구조가 바뀌면 아래 대안 셀렉터도 시도
    if not img_tags:
        img_tags = soup.select("div#comic_viewer img, div.viewer img, img")

    # 4) src 추출 (이미지 지연로딩 때문에 data-src에 있는 경우가 많음)
    img_urls = []
    for img in img_tags:
        src = (
            img.get("data-src")
            or img.get("data-original")
            or img.get("data-lazy-src")
            or img.get("src")
        )
        if not src:
            continue

        src = src.strip()
        full_url = urljoin(url, src)

        # 중복 제거
        if full_url not in img_urls:
            img_urls.append(full_url)

    if not img_urls:
        print("이미지 URL을 찾지 못했습니다. (페이지 구조 변경/접근 제한 가능)")
        return

    print(f"[{title} {no}회] 이미지 {len(img_urls)}개 다운로드 시작...")
    session = requests.Session()

    # 5) 이미지 다운로드
    for i, img_url in enumerate(img_urls, 1):
        ext = _guess_ext_from_url(img_url)
        filename = f"{i:03d}{ext}"
        filepath = save_dir / filename

        # 이미 다운로드된 파일이면 스킵
        if filepath.exists() and filepath.stat().st_size > 0:
            continue

        try:
            # 네이버 쪽은 referer가 있으면 안정적입니다.
            headers = dict(REQ_HEADER)
            headers["referer"] = url

            r = session.get(img_url, headers=headers, stream=True, timeout=30)
            r.raise_for_status()

            with open(filepath, "wb") as f:
                for chunk in r.iter_content(chunk_size=1024 * 128):
                    if chunk:
                        f.write(chunk)

            print(f"  - 저장: {filepath}")

        except Exception as e:
            print(f"  - 실패: {img_url}\n    이유: {e}")

    print(f"완료! 저장 폴더: {save_dir}")


In [22]:
download_one_episode(
    '고대동물기',
    49,
    'https://comic.naver.com/webtoon/detail?titleId=835910&no=49&week=sun'
)


[고대동물기 49회] 이미지 71개 다운로드 시작...
  - 저장: img/고대동물기/49/001.jpg
  - 저장: img/고대동물기/49/002.jpg
  - 저장: img/고대동물기/49/003.jpg
  - 저장: img/고대동물기/49/004.jpg
  - 저장: img/고대동물기/49/005.jpg
  - 저장: img/고대동물기/49/006.jpg
  - 저장: img/고대동물기/49/007.jpg
  - 저장: img/고대동물기/49/008.jpg
  - 저장: img/고대동물기/49/009.jpg
  - 저장: img/고대동물기/49/010.jpg
  - 저장: img/고대동물기/49/011.jpg
  - 저장: img/고대동물기/49/012.jpg
  - 저장: img/고대동물기/49/013.jpg
  - 저장: img/고대동물기/49/014.jpg
  - 저장: img/고대동물기/49/015.jpg
  - 저장: img/고대동물기/49/016.jpg
  - 저장: img/고대동물기/49/017.jpg
  - 저장: img/고대동물기/49/018.jpg
  - 저장: img/고대동물기/49/019.jpg
  - 저장: img/고대동물기/49/020.jpg
  - 저장: img/고대동물기/49/021.jpg
  - 저장: img/고대동물기/49/022.jpg
  - 저장: img/고대동물기/49/023.jpg
  - 저장: img/고대동물기/49/024.jpg
  - 저장: img/고대동물기/49/025.jpg
  - 저장: img/고대동물기/49/026.jpg
  - 저장: img/고대동물기/49/027.jpg
  - 저장: img/고대동물기/49/028.jpg
  - 저장: img/고대동물기/49/029.jpg
  - 저장: img/고대동물기/49/030.jpg
  - 저장: img/고대동물기/49/031.jpg
  - 저장: img/고대동물기/49/032.jpg
  - 저장: img/고대동물기/49/033.jpg
  - 저장: img/